In [ ]:
import sympy as sp
x,y,z = sp.symbols('x y z')
def grad(f):
    Gf = sp.Matrix([[f.diff(x),f.diff(y),f.diff(z)]])
    return Gf
def curl(f):
    Cf = sp.Matrix([[f[2].diff(y)-f[1].diff(z),f[0].diff(z)-f[2].diff(x),f[1].diff(x)-f[0].diff(y)]])
    return Cf
def div(f):
    return sp.Matrix([[f[0].diff(x)+f[1].diff(y)+f[2].diff(z)]])

def ev(f,ax,ay,az):
    return f.subs(x,ax).subs(y,ay).subs(z,az)


In [ ]:
import re
def cppcode(f):
    s = sp.ccode(f.simplify())
    s = re.sub("sin","autodiff::detail::sin",s)
    s = re.sub("cos","autodiff::detail::cos",s)
    s = re.sub("exp","autodiff::detail::exp",s)
    s = re.sub("pow","autodiff::detail::pow",s)
    return s
ADB = "      return autodiff1st(\n        [](autodiff::real x, autodiff::real y, autodiff::real z)->autodiff::real {\n          return "
ADE = ";\n        },X,dx,dy,dz);\n"
def ADPrint(A):
    if (A.shape == (1,1)):
        print(ADB,cppcode(A[0]),ADE,";",sep='')
    else:
        print("    if (i == 0) {\n",ADB,cppcode(A[0]),ADE,
              "    } else if (i == 1) {\n",ADB,cppcode(A[1]),ADE,
              "    } else {\n",ADB,cppcode(A[2]),ADE,
              "    };",sep='')


In [ ]:
def print0(f):
    df = grad(f)
    print("  static double f(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, unsigned j, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print1(f):
    df = curl(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2(f):
    df = div(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double df(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")

In [ ]:
def print1adj(f):
    df = -div(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double deltaf(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print2adj(f):
    df = curl(f)
    print("  static double f(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double deltaf(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")
def print3adj(f):
    df = -grad(f)
    print("  static double f(const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(f)
    print("  }\n  static double deltaf(unsigned i, const Eigen::Vector3d &X, unsigned dx, unsigned dy, unsigned dz) {")
    ADPrint(df)
    print("  }")

In [ ]:
P1C0 = sp.Matrix([[0,1,2]])
P1C1 = sp.Matrix([[(2*x**2-x+1)*(y**2+3*y-1)*(2*z**2-z+2),
                   (x**3-2*x**2-x+1)*(3*y-1)*(2*z**2-z+2),
                   (2*x**3+x**2-x+1)*(y**2+3*y-1)*(z+2)]])
P1C2 = sp.Matrix([[sp.sin(y)+sp.cos(x),
                   sp.exp(z),
                   2*x+1]])
P2C0 = sp.Matrix([[0,1,2]])
P2C1 = sp.Matrix([[(3*x**3+2*x**2-x+1)*(3*y-1)*(z+2),
                   (2*x**2-x+1)*(y**2+3*y-1)*(-z+2),
                   (x**2-x+1)*(y+1)*(2*z**2-z+2)]])
P2C2 = sp.Matrix([[sp.sin(y)+sp.cos(x),
                   sp.exp(z),
                   2*x+1]])
P3C0 = sp.Matrix([3*x/x])
P3C1 = sp.Matrix([57*x*y*z + 151*x*y + 2*x*z + 16*x + 72*y*z + 42*y - 80*z - 28])
P3C2 = sp.Matrix([sp.sin(y)*sp.cos(z)*x])

BB = x*(1-x)*y*(1-y)*z*(1-z)
P1H0 = sp.Matrix([[y*(1-y)*z*(1-z),
                   x*(1-x)*z*(1-z),
                   x*(1-x)*y*(1-y)]])
P2H0 = sp.Matrix([[x*(1-x),
                   y*(1-y),
                   z*(1-z)]])


In [ ]:
print("struct P1C0 {")
print1adj(P1C0)
print("};\nstruct P1C1 {")
print1adj(P1C1)
print("};\nstruct P1C2 {")
print1adj(P1C2)
print("};\nstruct P2C0 {")
print2adj(P2C0)
print("};\nstruct P2C1 {")
print2adj(P2C1)
print("};\nstruct P2C2 {")
print2adj(P2C2)
print("};\nstruct P3C0 {")
print3adj(P3C0)
print("};\nstruct P3C1 {")
print3adj(P3C1)
print("};\nstruct P3C2 {")
print3adj(P3C2)
print("};")

In [ ]:
print("struct P1A0 {")
print1adj(sp.hadamard_product(P1C2,P1H0))
print("};\nstruct P2A0 {")
print2adj(sp.hadamard_product(P2C2,P2H0))
print("};\nstruct P3A0 {")
print3adj(P3C2*BB)
print("};")

In [ ]:
print("struct P1A1 {")
print1adj(P1C0*BB)
print("};\nstruct P2A1 {")
print2adj(P2C0*BB)
print("};\nstruct P3A1 {")
print3adj(P3C0*BB)
print("};")